# MULTI-AGENT SYSTEM (IMPROVED)

This system is similar to the code_executor_chat.
The improvement will be on making tools available and adding two other agents:
- recommender; another LLM agent that needs to answer the user. In order to do that it can call other agents (coder and a function caller).
- function caller; This is a simple agent that can only call tools to retrieve data from, for example, wikipedia.


In [1]:
from autogen import AssistantAgent, GroupChatManager, GroupChat
from autogen import register_function, config_list_from_json
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent, TEXT_FORMATS
from pathlib import Path
import os

from CypherExecutor import CypherCodeExecutor
from tools.neo4j_tools import save_schema
from tools.tools import wiki_search

In [2]:
# Save database schema and then store the path to the directory
save_schema()
p = Path(__name__).parent.resolve() / "schemas"


#### LLM MODEL CONFIGURATION #####

config_list = config_list_from_json(env_or_file="CONFIG_LIST", filter_dict={"model": "gpt-4o"})
config_list[0]["api_key"] = os.environ.get("GITHUB_TOKEN")

llm_config  = {"config_list": config_list, "temperature": 0.2}

c_prompt = """
You are a helpful AI assistant.

### RULES ###
    - You have to explain everything you do.
    - When the question refers to something that is contained 
      in the database you must generate 1 query to retrieve data.
    - When generating a query you must use the Cypher format.

### DATABASE SCHEMA ### 
{input_context}
"""

r_prompt = """
You are a helpful AI assistant.

### RULES ###
  - You have to explain everything you do.
  - You have to use data from the database if it is related to the question.
    To retrieve them you need the help of the coder agent.
  - You have to use all the help from other assistant to answer precisely.

### TERMINATION ###
You can add 'TERMINATE' at the end of your message after the task is completed.
"""

# This are the accepted formats that can be retrieved by RetrieveUserProxy
print(f'Accepted formats for "docs_path": \n{TEXT_FORMATS}')

Accepted formats for "docs_path": 
['txt', 'json', 'csv', 'tsv', 'md', 'html', 'htm', 'rtf', 'rst', 'jsonl', 'log', 'xml', 'yaml', 'yml', 'pdf']


In [ ]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

recommender   = AssistantAgent(
    "recommender",
    llm_config=llm_config,  # Turn off LLM for this agent.
    is_termination_msg=termination_msg,
    system_message=r_prompt, 
    description="Assistant that gives tasks") 

doc_retriever = RetrieveUserProxyAgent(
    name="doc_retriever",
    is_termination_msg=termination_msg,
    max_consecutive_auto_reply=3,
    human_input_mode="NEVER",
    retrieve_config={
        "docs_path": str(p), # A list of urls, dirs or files can be passed
        "extra-docs": True,
        "model": config_list[0]["model"],
        "get_or_create": True,
        "customized_prompt": """CONTEXT:\n{input_context}\n\nQUESTION:\n{input_question}"""
    },
    code_execution_config=False,
    description="Assistant who has extra content retrieval power."
)

# Coder generates cypher queries with the help of the db schema 
coder      = AssistantAgent(
    name="coder",
    is_termination_msg=termination_msg,
    system_message=c_prompt,
    llm_config=llm_config,
    description="Coder creates cypher queries."
)

# Executor runs the query in a jupyter notebook
executor   = AssistantAgent(
    name="executor",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER", # THIS IS NOT SECURE! 
    description= "Executes queries.",
    code_execution_config={"executor": CypherCodeExecutor()}
    )

# Fun_caller calls a tool 
fun_caller = AssistantAgent(
    "fun_caller",
    llm_config=False,  # Turn off LLM for this agent.
    is_termination_msg=termination_msg,
    description="Executes tools.")


#### REGISTER FUNCTIONS ####

register_function(wiki_search,
                  caller= recommender,
                  executor= fun_caller,
                  description="Service that will retrieve information about the searched keyword.")

#############################

 
PROBLEM  = "Can you recommend me one action movie? "

def _reset_agents():
    """
    This function reset all the agents used for the group chat.
    This should be used every time you start a new conversation.
    """
    doc_retriever.reset()
    coder.reset()
    executor.reset()
    recommender.reset()

def state_transition(last_speaker, groupchat):
    """
    This function simply return the next speaker based on the last one.
    In this case we decided that doc_retriever should speak only to initiate chat
    and then the chat is from coder to executor in a sort of loop.
    """

    m = groupchat.messages[-1]
    
    if last_speaker in [fun_caller, doc_retriever]: return recommender
    elif last_speaker is executor: return coder 
    elif last_speaker is recommender: 
        if "tool_calls" in m: return fun_caller
        else: return coder
    
    elif last_speaker is coder:
        if "```cypher" in m["content"]: return executor
        else: return recommender
        
def rag_chat():
    """
    Resets the agents, creates the group chat and then initiate chat.
    The first agent to speak is the doc_retriever that will only pass:
        - prompt
        - files (like the schema), that is the context
        - question
    """
    _reset_agents()
    groupchat = GroupChat(agents=[doc_retriever, recommender, fun_caller, coder, executor], messages=[], 
                          max_round=10, speaker_selection_method=state_transition)
    manager   = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    doc_retriever.initiate_chat(manager,message=doc_retriever.message_generator, problem=PROBLEM,n_results=3)


In [4]:
#### START OF THE GROUP CHAT ####
rag_chat()

2024-09-30 12:12:08,224 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `autogen-docs`.


Trying to create collection.


2024-09-30 12:12:09,146 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 1 chunks.


VectorDB returns doc_ids:  [['c99bc2e0', '985292cd', 'ca5f356a']]
Adding content of doc c99bc2e0 to context.
Adding content of doc 985292cd to context.
Adding content of doc ca5f356a to context.
doc_retriever (to chat_manager):

CONTEXT:
[
    {
        "nodes": [
            {
                "name": "Movie",
                "indexes": [],
                "constraints": [
                    "Constraint( id=3, name='movieId', type='UNIQUENESS', schema=(:Movie {id}), ownedIndex=2 )"
                ],
                "properties": [
                    "movieId",
                    "plot",
                    "title"
                ]
            },
            {
                "name": "Person",
                "indexes": [],
                "constraints": [
                    "Constraint( id=5, name='personId', type='UNIQUENESS', schema=(:Person {id}), ownedIndex=4 )"
                ],
                "properties": [
                    "name",
                    "tmdbId"
       

[]

executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 

--------------------------------------------------------------------------------

Next speaker: coder

coder (to chat_manager):

It seems there was no specific output from the query, which might mean there are no movies with "action" directly in their plot or title in the database.

If you have any other criteria or need further assistance, feel free to ask!

--------------------------------------------------------------------------------

Next speaker: recommender

recommender (to chat_manager):

It looks like there were no direct matches for "action" in the movie titles or plots from the database. If you have any other preferences or need further recommendations, please let me know!

--------------------------------------------------------------------------------

Next speaker: coder

coder (to chat_manager):

If you have any specific actors, directors, or themes in mind, I can help find a movie that matche